In [1]:
from transformers import AutoModel, AutoTokenizer
import torch 
from peft import LoraConfig, get_peft_model, TaskType

In [2]:
checkpoint = "./chatglm-6b"
model = AutoModel.from_pretrained(checkpoint, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(checkpoint, trust_remote_code=True)

Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


In [3]:
def load_lora_config(model):
    config = LoraConfig(task_type=TaskType.CAUSAL_LM,
                        inference_mode=False,
                        r=8,
                        lora_alpha=32,
                        lora_dropout=0.1,
                        target_modules=["query_key_value"])
    return get_peft_model(model, config)


model = load_lora_config(model)

/usr/local/lib/python3.8/dist-packages/peft/tuners/lora.py:173: UserWarning: fan_in_fan_out is set to True but the target module is not a Conv1D. Setting fan_in_fan_out to False.
  warnings.warn(


In [4]:
model.load_state_dict(torch.load(f"fituned-chatglm6b-20230420/chatglm-6b-lora.pt"), strict=False)

_IncompatibleKeys(missing_keys=['base_model.model.transformer.word_embeddings.weight', 'base_model.model.transformer.layers.0.input_layernorm.weight', 'base_model.model.transformer.layers.0.input_layernorm.bias', 'base_model.model.transformer.layers.0.attention.query_key_value.weight', 'base_model.model.transformer.layers.0.attention.query_key_value.bias', 'base_model.model.transformer.layers.0.attention.dense.weight', 'base_model.model.transformer.layers.0.attention.dense.bias', 'base_model.model.transformer.layers.0.post_attention_layernorm.weight', 'base_model.model.transformer.layers.0.post_attention_layernorm.bias', 'base_model.model.transformer.layers.0.mlp.dense_h_to_4h.weight', 'base_model.model.transformer.layers.0.mlp.dense_h_to_4h.bias', 'base_model.model.transformer.layers.0.mlp.dense_4h_to_h.weight', 'base_model.model.transformer.layers.0.mlp.dense_4h_to_h.bias', 'base_model.model.transformer.layers.1.input_layernorm.weight', 'base_model.model.transformer.layers.1.input_la

In [6]:
model.half().cuda().eval()
response, history = model.chat(tokenizer, "核酸检测机构需要什么资质", history=[])
print(response)

市卫健委将审核申报资料，并结合市区新型冠状病毒核酸检测需求进行综合评估。经评估，具备相应资质和条件的，纳入我市开展新型冠状病毒核酸检测机构名单，并通知进行新型冠状病毒实验活动备案。
